In [496]:
import numpy as np
import  pandas as pd
import matplotlib.pyplot as plt

In [497]:
data= pd.read_excel("assignment6.xlsx",sheet_name="Sheet1")

In [498]:
data

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,7.0,3.2,4.7,1.4,versicolor
6,6.4,3.2,4.5,1.5,versicolor
7,6.9,3.1,4.9,1.5,versicolor
8,5.5,2.3,4.0,1.3,versicolor
9,6.5,2.8,4.6,1.5,versicolor


In [499]:
#seperating data and target
X=data.iloc[:,0:4]
Y=data.iloc[:,4]

In [500]:
#Calculate the euclidean distance between two points
def euclidean_distance(a,b):   
    point_a = np.array(a)
    point_b = np.array(b)
    distance = np.linalg.norm(point_a - point_b)
    
    return distance   

In [501]:
# print(X.iloc[0])
# print(X.iloc[1])

In [502]:
#splitting train and test set
def tt_split(X, y, test_size=0.25):

    i = int((1 - test_size) * X.shape[0]) 
    o = np.random.permutation(X.shape[0])
    
    X_train, X_test = np.split(np.take(X,o,axis=0), [i])
    y_train, y_test = np.split(np.take(y,o), [i])
    
    return X_train, X_test, y_train, y_test

In [503]:
X_train, X_test, y_train, y_test=tt_split(X,Y)

In [504]:
# print(X_train)
# print(X_test)
# print(y_train)
# print(y_test)

In [505]:
#scaling the data based on train data set
#calculate column means
def column_means(dataset):
    means = [0 for i in range(len(dataset.columns))]
    for i in range(len(dataset.columns)):
        col_values =dataset.iloc[:,i]
        means[i] = sum(col_values) / float(len(dataset.iloc[:,i]))
    return means

# calculate column standard deviations
def column_stdevs(dataset, means):
    stdevs = [0 for i in range(len(dataset.columns))]
    for i in range(len(dataset.columns)):
        col_values =dataset.iloc[:,i]
        variance = [(row-means[i])**2 for row in col_values]
        stdevs[i] = sum(variance)
    
    stdevs = [(x/(float(len(dataset.iloc[:,i])-1)))**0.5 for x in stdevs]
    return stdevs

# standardize dataset
def standardize_dataset(dataset, means, stdevs):
    for row in range(len(dataset)):
        for i in range(len(dataset.iloc[row,:])):
            dataset.iloc[row,i] = (dataset.iloc[row,i] - means[i]) / stdevs[i]

In [506]:
# Standardize dataset
dataset = X_train

# Estimate mean and standard deviation
means = column_means(dataset)
stdevs = column_stdevs(dataset, means)

standardize_dataset(dataset, means,stdevs)


In [507]:
# Standardize test dataset based on train dataset 
standardize_dataset(X_test, means,stdevs)

In [508]:
def knn_predict(X_train,X_test,y_train,y_test,k):
    from collections import Counter
    
    y_hat_test=[]
    
    for test_point in list(X_test.index):
        distances=[]
        
        for train_point in list(X_train.index):
            distance=euclidean_distance(X_test.loc[test_point],X_train.loc[train_point])
            distances.append(distance)
        
        #converting distances in dataframe
        df_dists =pd.DataFrame(data=distances,index=y_train.index,columns=['dist'])
        
        #sorting distances
        df_nn=df_dists.sort_values(by=['dist'],axis=0)[:k]
        
        #counter object to track the labels of k most near neighbours
        counter=Counter(y_train[df_nn.index])
        
        #select the most commong label
        prediction = counter.most_common()[0][0]
        
        y_hat_test.append(prediction)
        
    return y_hat_test

In [509]:
y_hat_test=knn_predict(X_train, X_test, y_train, y_test,k=3)

In [513]:
print("Actual : ")
print(list(y_test))
print("Predicted : ")
print(y_hat_test)

Actual : 
['versicolor', 'versicolor', 'setosa']
Predicted : 
['versicolor', 'versicolor', 'setosa']


In [514]:
#for some unknown observations checking with different k value
X_test1=pd.DataFrame( {'sepal_length': [8.2],
                       'sepal_width': [2.6],
                       'petal_length': [3.2],
                       'petal_width': [1.8]
                      } )
y_test1=pd.DataFrame([""])

In [515]:
standardize_dataset(X_test1, means,stdevs)

In [516]:
y_hat_test_3=knn_predict(X_train.append(X_test), X_test1, y_train.append(y_test),y_test1 ,k=3)
y_hat_test_3

['versicolor']

In [517]:
y_hat_test_5=knn_predict(X_train.append(X_test), X_test1, y_train.append(y_test),y_test1 ,k=5)
y_hat_test_5

['versicolor']

In [518]:
y_hat_test_7=knn_predict(X_train.append(X_test), X_test1, y_train.append(y_test),y_test1 ,k=7)
y_hat_test_7

['versicolor']